In [1]:
import numpy as np
import pandas as pd
import sqlite3
import pickle

import adastate

with open("../adagrid/4d_full/3880.pkl", "rb") as f:
    S = pickle.load(f)
# all_data = np.concatenate((S.db.data, S.sim_sizes[:, None], S.todo[:, None], S.g.grid_pt_idx[:, None], S.g.null_truth), axis=1)
# np.save('2090.npy', all_data)

In [2]:
df = pd.DataFrame(S.db.data)

In [3]:
rename_dict = dict()
for k, v in S.db.slices.items():
    if isinstance(v, slice):
        for i in range(v.start, v.stop, 1 if v.step is None else v.step):
            rename_dict[i] = k + "_" + str(i - v.start)
    else:
        rename_dict[v] = k
df.rename(columns=rename_dict, inplace=True)

df["sim_sizes"] = S.sim_sizes
df["todo"] = S.todo
df["grid_pt_idx"] = S.g.grid_pt_idx
for d in range(S.g.d):
    df[f"theta_{d}"] = S.g.thetas[S.g.grid_pt_idx, d]
    df[f"radii_{d}"] = S.g.radii[S.g.grid_pt_idx, d]

for i in range(S.g.null_truth.shape[1]):
    df[f"null_truth_{i}"] = S.g.null_truth[:, i]

In [4]:
df.head()

,alpha0,orig_lam,B_lam_0,B_lam_1,B_lam_2,B_lam_3,B_lam_4,B_lam_5,B_lam_6,B_lam_7,...,radii_0,theta_1,radii_1,theta_2,radii_2,theta_3,radii_3,null_truth_0,null_truth_1,null_truth_2
0,0.011102,0.102584,0.114941,0.104812,0.099318,0.104231,0.099771,0.106665,0.103630,0.100998,...,0.015625,-0.046875,0.015625,-0.640625,0.015625,-0.859375,0.015625,True,True,True
1,0.011101,0.101231,0.115902,0.104259,0.096866,0.102761,0.097951,0.105919,0.101904,0.099771,...,0.015625,-0.046875,0.015625,-0.671875,0.015625,-0.828125,0.015625,True,True,True
2,0.011111,0.097951,0.111798,0.101231,0.093288,0.099771,0.094733,0.103630,0.099771,0.096866,...,0.015625,-0.046875,0.015625,-0.703125,0.015625,-0.828125,0.015625,True,True,True
3,0.011123,0.097845,0.110511,0.100481,0.092762,0.099318,0.094563,0.102100,0.099318,0.096798,...,0.015625,-0.046875,0.015625,-0.703125,0.015625,-0.859375,0.015625,True,True,True
4,0.011102,0.102584,0.114941,0.104812,0.099318,0.104231,0.099771,0.106665,0.103630,0.100998,...,0.015625,-0.046875,0.015625,-0.640625,0.015625,-0.859375,0.015625,True,True,True


In [5]:
chunk = df[:100000]

## SQLite

In [6]:
con = sqlite3.connect("tutorial.db")

In [7]:
%%time
chunk.to_sql("tiles", con, if_exists="replace")

CPU times: user 708 ms, sys: 361 ms, total: 1.07 s
Wall time: 1.47 s


100000

In [8]:
con.execute("select count(*) from tiles").fetchall()

[(100000,)]

In [9]:
%%time
dfload = pd.read_sql("select * from tiles", con)

CPU times: user 2.24 s, sys: 122 ms, total: 2.36 s
Wall time: 2.37 s


In [10]:
dfload.head()

,index,alpha0,orig_lam,B_lam_0,B_lam_1,B_lam_2,B_lam_3,B_lam_4,B_lam_5,B_lam_6,...,radii_0,theta_1,radii_1,theta_2,radii_2,theta_3,radii_3,null_truth_0,null_truth_1,null_truth_2
0,0,0.011102,0.102584,0.114941,0.104812,0.099318,0.104231,0.099771,0.106665,0.103630,...,0.015625,-0.046875,0.015625,-0.640625,0.015625,-0.859375,0.015625,1,1,1
1,1,0.011101,0.101231,0.115902,0.104259,0.096866,0.102761,0.097951,0.105919,0.101904,...,0.015625,-0.046875,0.015625,-0.671875,0.015625,-0.828125,0.015625,1,1,1
2,2,0.011111,0.097951,0.111798,0.101231,0.093288,0.099771,0.094733,0.103630,0.099771,...,0.015625,-0.046875,0.015625,-0.703125,0.015625,-0.828125,0.015625,1,1,1
3,3,0.011123,0.097845,0.110511,0.100481,0.092762,0.099318,0.094563,0.102100,0.099318,...,0.015625,-0.046875,0.015625,-0.703125,0.015625,-0.859375,0.015625,1,1,1
4,4,0.011102,0.102584,0.114941,0.104812,0.099318,0.104231,0.099771,0.106665,0.103630,...,0.015625,-0.046875,0.015625,-0.640625,0.015625,-0.859375,0.015625,1,1,1


In [11]:
%%time
con.execute(
    "select " + ",".join([f"min(B_lam_{i})" for i in range(50)]) + " from tiles"
).fetchall()

CPU times: user 87.2 ms, sys: 18.8 ms, total: 106 ms
Wall time: 107 ms


[(0.06656096130609512,
  0.064506895840168,
  0.06400763243436813,
  0.06433851271867752,
  0.06519397348165512,
  0.06536728888750076,
  0.06479469686746597,
  0.0653068944811821,
  0.06488805264234543,
  0.06114932522177696,
  0.06371963024139404,
  0.06594660133123398,
  0.06602223962545395,
  0.06479965895414352,
  0.06421804428100586,
  0.06102116033434868,
  0.06707166880369186,
  0.06463774293661118,
  0.06417149305343628,
  0.06558826565742493,
  0.062491193413734436,
  0.06430821120738983,
  0.06347959488630295,
  0.06500014662742615,
  0.06350678950548172,
  0.06543929874897003,
  0.06388048827648163,
  0.06387448310852051,
  0.06388067454099655,
  0.06731399148702621,
  0.0632510855793953,
  0.06611113250255585,
  0.06383296102285385,
  0.06515980511903763,
  0.06482897698879242,
  0.06717406213283539,
  0.06524978578090668,
  0.06498073786497116,
  0.06551919877529144,
  0.06509511917829514,
  0.06463774293661118,
  0.05955151468515396,
  0.06393741816282272,
  0.0652446895

## duckdb

In [3]:
import pandas as pd

df = pd.read_parquet("dbtest.parquet")

In [6]:
chunk = df[:]
chunk.shape

(19408289, 69)

In [7]:
import duckdb
import pyarrow as pa
import pickle

con = duckdb.connect()

In [8]:
%%time
# con.execute('drop table tiles')
in_tbl = pa.Table.from_pandas(chunk)
con.execute("create table tiles as select * from chunk")
# con.execute('create index twb_min_lam_idx on tiles (twb_min_lam)')

CPU times: user 15.6 s, sys: 2.96 s, total: 18.6 s
Wall time: 3.24 s


In [66]:
%%time
out = con.execute("select * from tiles order by twb_min_lam limit 10000").fetch_df()
out.head()

CPU times: user 731 ms, sys: 181 ms, total: 912 ms
Wall time: 178 ms


,alpha0,orig_lam,B_lam_0,B_lam_1,B_lam_2,B_lam_3,B_lam_4,B_lam_5,B_lam_6,B_lam_7,...,radii_0,theta_1,radii_1,theta_2,radii_2,theta_3,radii_3,null_truth_0,null_truth_1,null_truth_2
0,0.016982,0.065629,0.066356,0.065826,0.066355,0.063562,0.065653,0.070084,0.066393,0.066176,...,0.007812,0.679688,0.007812,0.679688,0.007812,-0.757812,0.007812,True,True,True
1,0.016982,0.065629,0.066356,0.065826,0.066355,0.063562,0.065653,0.070084,0.066393,0.066176,...,0.007812,0.679688,0.007812,0.679688,0.007812,-0.757812,0.007812,True,True,True
2,0.016982,0.065629,0.066356,0.065826,0.066355,0.063562,0.065653,0.070084,0.066393,0.066176,...,0.007812,0.679688,0.007812,0.679688,0.007812,-0.757812,0.007812,True,True,True
3,0.016982,0.065629,0.066356,0.065826,0.066355,0.063562,0.065653,0.070084,0.066393,0.066176,...,0.007812,0.679688,0.007812,0.679688,0.007812,-0.757812,0.007812,True,True,True
4,0.016982,0.065629,0.066356,0.065826,0.066355,0.063562,0.065653,0.070084,0.066393,0.066176,...,0.007812,0.679688,0.007812,0.679688,0.007812,-0.757812,0.007812,True,True,True


In [71]:
%%time
con.execute(
    "select " + ",".join([f"min(B_lam_{i})" for i in range(50)]) + " from tiles"
).fetchall()

CPU times: user 4.42 s, sys: 207 ms, total: 4.63 s
Wall time: 561 ms


[(0.06231215223670006,
  0.055375631898641586,
  0.0625298023223877,
  0.061695247888565063,
  0.0625298023223877,
  0.060130525380373,
  0.05916070193052292,
  0.056377604603767395,
  0.05082782730460167,
  0.05162762105464935,
  0.05911017581820488,
  0.06067570298910141,
  0.06466516107320786,
  0.05746819078922272,
  0.06012817844748497,
  0.051577892154455185,
  0.05938999727368355,
  0.06441992521286011,
  0.05864203721284866,
  0.05549140274524689,
  0.06099971383810043,
  0.05746819078922272,
  0.05916070193052292,
  0.0575517937541008,
  0.062339331954717636,
  0.05890149250626564,
  0.060382794588804245,
  0.061473529785871506,
  0.06222115457057953,
  0.06067570298910141,
  0.05864203721284866,
  0.06321893632411957,
  0.06258217245340347,
  0.06157900020480156,
  0.05329404026269913,
  0.0628439337015152,
  0.061255715787410736,
  0.06314831227064133,
  0.06263559311628342,
  0.0528295561671257,
  0.06314831227064133,
  0.05884512886404991,
  0.05989359691739082,
  0.058642